In [1]:
from migen import *
from migen.genlib.fsm import FSM
from migen.genlib.misc import WaitTimer

class HBridgeDriver(Module):
    def __init__(self, delay):
        
        # inputs
        self.d = Signal()  # coil direction
        self.en = Signal() # enable switching of direction
        
        # outputs
        # signals driving relay on/off; will need to be wired to physical pin
        # in platform definition/top-level build
        self.r0 = Signal(reset=0)
        self.r1 = Signal(reset=1)
        self.r2 = Signal()
        self.r3 = Signal()
        
        self.delay = delay # store attribute for testbench...?

        # declaring these here is a pattern for doing, eg, the following in platform build:
        #    top.comb += [platform.request("ttl").eq(o) for o in top.outputs]
        # (where `top` is an instance of HBridgeDriver)
        self.inputs = [self.en, self.d]
        self.outputs = [self.r0, self.r1, self.r2, self.r3]
        
        # # #
        
        # this syntax exposes these submodules as self.fsm, self.timer as well
        self.submodules.fsm = fsm = FSM(reset_state="LOCKOUT")
        self.submodules.timer = timer = WaitTimer(delay)        
        
        # state machine
        # "LOCKOUT" -> do not allow coil direction change. Exit this state
        #              conditioned on zero current, plus some "delay" for ringdown
        # "ENABLE" -> do allow coil direction to swap
        fsm.act("LOCKOUT",
            If(self.en,
                timer.wait.eq(1)
            ).Else(timer.wait.eq(0)),
            If(timer.done, NextState("ENABLE"))
        )
        
        fsm.act("ENABLE",
            If(self.en,
                self.r0.eq(self.d),
                self.r1.eq(~self.d)
            ).Else(NextState("LOCKOUT"))
        )
        
        # will need to check polarity here for relay connections to 
        # get proper h-bridge behavior...
        self.comb += [self.r2.eq(self.r0), self.r3.eq(self.r1)]

First, let's run a testbench!

In [3]:
dut = HBridgeDriver(delay=10)

def tb():
    # drive inputs at start of simulation
    yield dut.d.eq(0)
    yield dut.en.eq(0)
    
    # check that r0, r1 are initialized properly
    assert (yield dut.r0) == 0
    assert (yield dut.r1) == 1
    yield
    
    # enable switching & switch direction concurrently
    yield dut.en.eq(1)
    yield dut.d.eq(1)
    for _ in range(dut.delay + 1):
        # check that r0, r1 don't change until after delay lockout period
        assert (yield dut.r0) == 0
        assert (yield dut.r1) == 1
        yield
    
    yield # need extra clock for state machine transition?
    
    # check that r0, r1 have swapped polarity
    assert (yield dut.r0) == 1
    assert (yield dut.r1) == 0

run_simulation(dut, tb(), vcd_name="HBridgeDriver_tb.vcd")

Here's what the waveform looks like:

![HBridgeDriver](HBridgeDriver_tb_gtkwave.png)

Ok, next step is to instatitate a particular hardware platform, and build this module in gateware.

Going to do now for pipistrello, but ultimately want to port this to CPLD (eg, this [Digilent Board](http://store.digilentinc.com/cmod-c2-breadboardable-coolrunner-ii-cpld-module/) or this similar but [cheaper one from seeed](https://www.seeedstudio.com/XC2C64A-CoolRunner-II-CPLD-development-board-p-800.html).

Unfortunately I don't have ISE installed, so it errors out, but the generated .v module looks close to appropriate...

In [ ]:
from migen.build.platforms import pipistrello
from migen.build.generic_platform import *

# generic connector from old artiq target file
_pipistrello_io = [
    ("ttl", 0, Pins("C:11"), IOStandard("LVTTL")),
    ("ttl", 1, Pins("C:10"), IOStandard("LVTTL")),
    ("ttl", 2, Pins("C:9"), IOStandard("LVTTL")),
    ("ttl", 3, Pins("C:8"), IOStandard("LVTTL")),
    ("ttl", 4, Pins("C:7"), IOStandard("LVTTL")),
    ("ttl", 5, Pins("C:6"), IOStandard("LVTTL")),
    ("ttl", 6, Pins("C:5"), IOStandard("LVTTL")),
    ("ttl", 7, Pins("C:4"), IOStandard("LVTTL"))
]


def build_platform(delay=10):
    platform = pipistrello.Platform()
    
    # must `add_extension` to be able to request IO
    platform.add_extension(_pipistrello_io)

    top = HBridgeDriver(delay)

    # drive direction, enable signals from first two TTL
    # alternatively, could do platform.request("ttl", 0), ... to get explicit
    top.comb += [
        top.en.eq(platform.request("ttl")),
        top.d.eq(platform.request("ttl"))
    ]

    # drive output pins
    # (general) QUESTION: do you have to explicitly declare in vs out? or is it inferred
    # from reg vs wire...?
    top.comb += [platform.request("ttl").eq(o) for o in top.outputs]

    platform.build(top, build_dir="pipistrello_hbridge", build_name="hbridge_top")
